# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [65]:
import os
import json
from openai import OpenAI
import gradio as gr
from products import Product, Category, ProductCatalog

In [66]:
MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [67]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [68]:
SYSTEM_PROMPT = "Eres un assistente de AI que ayuda en ventas para productos medicos de la empresa Medical Group Occidente. "
SYSTEM_PROMPT += "Responde amablemente y con un tono cercano al humano. "
SYSTEM_PROMPT += "Si no conoces la respuesta, di que no lo sabes. "
SYSTEM_PROMPT += "No inventes respuestas, siempre responde con la información que tengas a la mano. "
SYSTEM_PROMPT += "Si el usuario te pide algo que no está relacionado con productos medicos, recuérdale que eres un asistente de ventas para productos medicos."
SYSTEM_PROMPT += """ 
Tienes acceso a un catálogo completo de productos que incluye:
- Desechables (guantes, cubrebocas, etc.)
- Desinfección (alcohol, toallitas desinfectantes)
- Electrodos (ECG adultos y pediátricos)
- Limpieza (jabones, detergentes)
- Equipamiento (estetoscopios, tensiómetros)
- Consumibles (jeringas, agujas)

Puedes responder preguntas sobre:
- Precios en pesos mexicanos (MXN)
- Disponibilidad de productos
- Especificaciones técnicas
- Categorías de productos
- Descripciones detalladas

Siempre responde en español y proporciona información precisa sobre los productos.
Si no tienes información sobre un producto, responde que no lo conoces. Pero que si dejan su correo,
o numero de telefono, te contactaras con ellos.
"""


In [69]:
# Medical Group Occidente Product Prices List
# List of products with their prices
catalog = ProductCatalog()

In [70]:
# Define the tools/functions that the model can use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_products_by_category",
            "description": "Obtiene todos los productos de una categoría específica",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "enum": [cat.value for cat in Category],
                        "description": "Categoría del producto"
                    }
                },
                "required": ["category"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_products",
            "description": "Busca productos por nombre o descripción",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Término de búsqueda"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_all_products",
            "description": "Obtiene todos los productos del catálogo",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_product_by_name",
            "description": "Obtiene un producto por su nombre",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del producto"
                    }
                }
            }
        }
    }
]

def format_product_info(product: Product) -> str:
    return f"""
    Nombre: {product.name}
    Categoría: {product.category.value}
    Precio: ${product.price:.2f} MXN
    Descripción: {product.description}
    Stock disponible: {product.stock}
    """

def get_products_by_category(category: str) -> str:
    try:
        # Find the category enum by its value
        category_enum = next((cat for cat in Category if cat.value == category), None)
        if category_enum is None:
            available_categories = [cat.value for cat in Category]
            return f"Categoría '{category}' no válida. Categorías disponibles: {', '.join(available_categories)}"
        
        products = catalog.get_products_by_category(category_enum)
        if not products:
            return f"No se encontraron productos en la categoría {category}"
        
        return "\n".join([format_product_info(p) for p in products])
    except Exception as e:
        return f"Error al buscar productos: {str(e)}"

def search_products(query: str) -> str:
    try:
        products = catalog.search_product(query)
        if not products:
            return f"No se encontraron productos que coincidan con '{query}'"
        
        return "\n".join([format_product_info(p) for p in products])
    except Exception as e:
        return f"Error al buscar productos: {str(e)}"

def get_all_products() -> str:
    try:
        products = catalog.get_all_products()
        return "\n".join([format_product_info(p) for p in products])
    except Exception as e:
        return f"Error al obtener productos: {str(e)}"

def get_product_by_name(name: str) -> str:
    try:
        product = catalog.get_product_by_name(name)
        if not product:
            return f"No se encontró el producto '{name}'"
        
        return format_product_info(product)
    except Exception as e:
        return f"Error al obtener el producto: {str(e)}"



In [73]:
def chat(user_input_text: str, history: list) -> list:
    print(f"user_input_text: {user_input_text}")
    try:
        # List to build messages for the OpenAI API call
        api_messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Add existing conversation history to the API messages
        for human_message, ai_message in history:
            if human_message is not None: # Should always be a string from user
                api_messages.append({"role": "user", "content": str(human_message)})
                print(f"human_message: {human_message}")
            if ai_message is not None: # AI message might be None if there was an error previously
                api_messages.append({"role": "assistant", "content": str(ai_message)})
                print(f"ai_message: {ai_message}")

        # Add the current user message to the API messages
        api_messages.append({"role": "user", "content": user_input_text})

        # First API call to get a response (might include tool calls)
        response = openai.chat.completions.create(
            model=MODEL,
            messages=api_messages,
            tools=tools,
            tool_choice="auto"  # Let the model decide if it needs to use tools
        )

        # Get the assistant's response message object
        assistant_message_object = response.choices[0].message

        bot_response_content = "" # Initialize

        print(f"assistant_message_object: {assistant_message_object}")
        
        if assistant_message_object.tool_calls:
            # If there are tool calls, we need to process them
            api_messages.append(assistant_message_object)  # Add assistant's tool request to API history

            for tool_call in assistant_message_object.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                function_response_str = ""
                if function_name == "get_products_by_category":
                    function_response_str = get_products_by_category(category=function_args.get("category"))
                elif function_name == "search_products":
                    function_response_str = search_products(query=function_args.get("query"))
                elif function_name == "get_all_products":
                    function_response_str = get_all_products()
                else:
                    function_response_str = f"Unknown function: {function_name}"

                api_messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_response_str,
                })
            
            # Second API call after processing tools
            final_response = openai.chat.completions.create(
                model=MODEL,
                messages=api_messages  # Send the full conversation including tool responses
            )
            bot_response_content = final_response.choices[0].message.content
        else:
            # No tool calls, use the content from the first response
            bot_response_content = assistant_message_object.content

        # Ensure the bot's response is a string, even if it's None or empty
        if bot_response_content is None:
            bot_response_content = ""

        # Append the new interaction (user input string, bot response string) to history
        history.append([user_input_text, bot_response_content])
        return history

    except Exception as e:
        error_message_for_user = f"Lo siento, ocurrió un error inesperado: {str(e)}"
        # Append the user's message and the error to history for display
        history.append([user_input_text, error_message_for_user])
        return history

In [38]:
# CSS Styles
# Custom CSS for better styling
custom_css = """
#chatbot {
    height: 600px;
    overflow-y: auto;
    border: 1px solid #e0e0e0;
    border-radius: 10px;
    padding: 20px;
    background-color: #f9f9f9;
}

#chatbot .message {
    padding: 10px;
    margin: 5px 0;
    border-radius: 5px;
}

#chatbot .user-message {
    background-color: #e3f2fd;
}

#chatbot .bot-message {
    background-color: #f5f5f5;
}

#input-box {
    border-radius: 20px;
    padding: 10px 20px;
    border: 1px solid #e0e0e0;
}

#send-button {
    border-radius: 20px;
    background-color: #2196f3;
    color: white;
}

#examples {
    background-color: #f5f5f5;
    padding: 20px;
    border-radius: 10px;
    margin-top: 20px;
    color: black;
}

#examples h3 {
    color: #2196f3;
    margin-bottom: 10px;
}

#examples ul li {
    color: black;
}
"""

In [74]:
# Create Gradio interface
with gr.Blocks(css=custom_css) as ui:
    gr.Markdown("""
    # 🏥 Asistente de Productos Médicos
    
    Bienvenido al asistente virtual de productos médicos. Puedo ayudarte a encontrar información sobre nuestros productos, precios y disponibilidad.
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                elem_id="chatbot",
                label="Chat",
                height=600
            )
            with gr.Row():
                msg = gr.Textbox(
                    elem_id="input-box",
                    placeholder="Escribe tu pregunta aquí...",
                    label="",
                    scale=4
                )
                send = gr.Button(
                    "Enviar",
                    elem_id="send-button",
                    scale=1
                )
        
        with gr.Column(scale=1):
            gr.Markdown("""
            ### 📝 Ejemplos de preguntas:
            
            - ¿Cuánto cuestan los guantes de látex?
            - ¿Qué productos hay en la categoría de desinfección?
            - ¿Tienen estetoscopios en stock?
            - ¿Cuál es la descripción del tensiómetro digital?
            - ¿Qué productos hay disponibles en la categoría de consumibles?
            - ¿Cuál es el precio del alcohol en gel?
            """, elem_id="examples")
    
    # Set up event handlers
    msg.submit(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=chatbot
    )
    
    send.click(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=chatbot
    )

# Launch the interface
ui.launch(inbrowser=True, share=True)

/var/folders/hs/fjgqy_d902bfb97f4dvc2frm0000gn/T/ipykernel_7634/1456721150.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7878
* Running on public URL: https://765862eb4bf176b7a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


user_input_text: Que productos manejeas que sean baratos 
assistant_message_object: ChatCompletionMessage(content='Para poder ayudarte mejor, ¿podrías especificar qué tipo de productos te interesan? Tenemos diferentes categorías como desechables, desinfección, electrodos, limpieza, equipamiento y consumibles. Así podré buscar los productos más económicos en la categoría que elijas.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
user_input_text: equipamentos
human_message: Que productos manejeas que sean baratos 
ai_message: Para poder ayudarte mejor, ¿podrías especificar qué tipo de productos te interesan? Tenemos diferentes categorías como desechables, desinfección, electrodos, limpieza, equipamiento y consumibles. Así podré buscar los productos más económicos en la categoría que elijas.
assistant_message_object: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_c